## 1. Carga de datos
#### Filtro Madrid y elimino variables no relevantes

In [1]:
import numpy  as np  
import pandas as pd

In [ ]:
#Cargo fichero
raw_data = pd.read_csv("airbnb-listings-extract.csv", sep=';')
print(raw_data.shape)
raw_data.head(5).T  #datos en bruto

#### Filtro y me quedo sólo con los datos correspondientes a Madrid.

In [75]:
# Análisis de España
total_filas = len(raw_data)
spain_count = raw_data[raw_data['Country'] == 'Spain'].shape[0]
spain_percentage = (spain_count / total_filas) * 100

print(f"Total dataset: {total_filas:,} propiedades")
print(f"Total España: {spain_count:,} propiedades ({spain_percentage:.2f}%)")

Total dataset: 14,780 propiedades
Total España: 14,001 propiedades (94.73%)


In [76]:
# Análisis de Madrid 
total_filas = len(raw_data)

# Buscar en Madrid
madrid_filter = raw_data['City'].str.contains('madrid', case=False, na=False)
madrid_count = madrid_filter.sum()
madrid_percentage = (madrid_count / total_filas) * 100

# Buscar otras ciudades de la Comunidad de Madrid
ciudades_madrid = [
    'pozuelo', 'alcalá', 'alcala', 'getafe', 'leganés', 'leganes', 'fuenlabrada', 
    'alcorcón', 'alcorcon', 'móstoles', 'mostoles', 'torrejón', 'torrejon',
    'parla', 'alcobendas', 'aranjuez', 'coslada', 'las rozas', 'majadahonda',
    'rivas', 'collado villalba', 'boadilla', 'san sebastián de los reyes',
    'san sebastian de los reyes', 'tres cantos', 'valdemoro', 'pinto',
    'arganda', 'chinchón', 'chinchon', 'navalcarnero', 'colmenar viejo'
]

otras_madrid_filter = raw_data['City'].str.contains('|'.join(ciudades_madrid), case=False, na=False) & ~madrid_filter
otras_madrid_count = otras_madrid_filter.sum()
otras_madrid_percentage = (otras_madrid_count / total_filas) * 100

# Total Comunidad de Madrid
total_madrid_count = madrid_count + otras_madrid_count
total_madrid_percentage = (total_madrid_count / total_filas) * 100

print(f"Total dataset: {total_filas:,} propiedades")
print(f"Total Madrid: {total_madrid_count:,} propiedades ({total_madrid_percentage:.2f}%)")

Total dataset: 14,780 propiedades
Total Madrid: 13,252 propiedades (89.66%)


Elegir Madrid supone perder cerca de un 11% de datos, lo cual es bastante dado el tamaño del dataset. Pero elegir como muestra España (y perder sólo 5%) puede estar metiendo más variabilidad al modelo para Madrid si los precios entre comunidades son muy diferentes y no habría suficiente muestra representativa del resto de ciudades

In [78]:
# Filtro Madrid
total_madrid_filter = madrid_filter | otras_madrid_filter

# Creo copia filtrada por Madrid 
raw_data_madrid = raw_data[total_madrid_filter].copy() #sólo madrid 

print(f"Dataset original: {raw_data.shape}")
print(f"Dataset Madrid: {raw_data_madrid.shape}")
print(f"Registros eliminados: {len(raw_data) - len(raw_data_madrid):,}")

Dataset original: (14780, 89)
Dataset Madrid: (13252, 89)
Registros eliminados: 1,528


#### Elimino variables no relevantes para caso estudio ya que no voy a hacer procesamiento de texto (Url's, Id's, descripciones y metadata). 
Además elimino las columnas `City`, `Country` por seleccionar sólo Madrid

In [80]:
# una copia de los datos para trabajar 
data_madrid= raw_data_madrid.copy()

columns_to_drop = [
    'ID', 'Listing Url', 'Scrape ID', 'Thumbnail Url', 'Medium Url', 
    'Picture Url', 'XL Picture Url', 'Host URL', 'Host Thumbnail Url', 
    'Host Picture Url', 'Last Scraped', 'Name', 'Summary', 'Space', 
    'Description', 'Neighborhood Overview', 'License', 'Jurisdiction Names', 
    'Cancellation Policy', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Host ID', 'Host Name', 
    'Host Location', 'Host About', 'Host Response Time', 'Geolocation', 'Smart Location', 'Market', 'Street', 
    'Host Verifications', 'Neighbourhood', 'Neighbourhood Cleansed', 'State',
    'Neighbourhood Group Cleansed', 'Country Code', 'Calendar Updated', 'City', 'Country',
    'Calendar last Scraped', 'Features', 'Host Neighbourhood'
]

input_madrid= data_madrid.drop(columns=columns_to_drop)  # Aplico la eliminación de variables

print(f"Dataset original: {raw_data.shape}")
print(f"Dataset después de limpieza: {input_madrid.shape}")
input_madrid.head(5).T #sólo madrid sin columnas texto

Dataset original: (14780, 89)
Dataset después de limpieza: (13252, 44)


,40,41,42,43,44
Experiences Offered,none,none,none,none,none
Host Since,2015-01-06,2015-01-01,2016-04-18,2012-09-08,2013-12-25
Host Response Rate,84.0,100.0,100.0,100.0,100.0
Host Acceptance Rate,NaN,NaN,NaN,NaN,NaN
Host Listings Count,3.0,1.0,2.0,8.0,2.0
Host Total Listings Count,3.0,1.0,2.0,8.0,2.0
Zipcode,28007,28001,28001,28001,28001
Latitude,40.406953,40.425904,40.428352,40.427454,40.428671
Longitude,-3.670894,-3.681647,-3.687155,-3.685763,-3.685423
Property Type,Apartment,Apartment,Apartment,Apartment,Apartment


## 2. División de datos Train / Test
#### Aplico método `train_test_split` para separación fisica en dos archivos .csv (datos de entrenamiento y prueba)

In [81]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(input_madrid, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('airbnb-listings-extract-train.csv', sep=';', decimal='.', index=False)
test.to_csv('airbnb-listings-extract-test.csv', sep=';', decimal='.', index=False)

Dimensiones del dataset de training: (10601, 44)
Dimensiones del dataset de test: (2651, 44)


In [ ]:
# Trabajo sólo con el dataset de train

pretrain_data = pd.read_csv('airbnb-listings-extract-train.csv', sep=';', decimal='.') #Train de sólo madrid sin columnas texto
pretrain_data.head(5).T

## 3. Análisis exploratorio
#### Miro los datos de Train para saber distribuciones estadisticas, outliers, correlaciones.
- info, describe...
- Histogramas, scatterplots..
- (Matriz correlaciones entre variables)
- (Método de filtrado: `f_reg, mutual_info_reg`)

In [90]:
# Clasificación de variables: Numéricas vs Categóricas
numerical_var = pretrain_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_var = pretrain_data.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"{len(pretrain_data.columns)} variables total | {len(numerical_var)} numéricas | {len(categorical_var)} categóricas")

44 variables total | 35 numéricas | 9 categóricas


### 3.1 Análisis exploratorio - Variables categórcias

Voy a estudiar primero las categoricas:

| Atributo | descripción | conclusión |
|:---------|:------------|:-----------|
| *Experiences Offered* | experiencias adicionales al piso | |
| *Host Since* | fecha registro del anfitrión en Airbnb | Pasar a días --> númerica y tratar nulos =0|
| *Host Acceptance Rate* | % solicitudes aceptadas por anfitrión | Eliminar por alto valor de nulos|
| *City* | ciudad | Filtrado previamente por Madrid: eliminar |
| *ZipCode* | código barras | Standby |
| *Country* | pais | Eliminar |
| *Property Type* | tipo de propiedad | |
| *Room Type* | tipo de habitación | |
| *Bed Type* | tipo de cama | |
| *Amenities* | comodidades adicionales | Standby |
| *Has Availability* | disponibilidad actual | Eliminar por alto valor de nulos|
| *First Review* | fecha primera reseña | Hacer la diferencia a dias con Last Review y tratar nulos=0|
| *Last Review* | fecha última reseña | 

In [ ]:
# Variables CATEGÓRICAS
categorical_data = []
categorical_var = pretrain_data.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_var:
    null_pct = (pretrain_data[col].isnull().sum() / len(pretrain_data)) * 100
    categorical_data.append({
        'Variable': col,
        'Dtype': str(pretrain_data[col].dtype),
        'Valores_únicos': pretrain_data[col].nunique(),
        'Nulos_%': round(null_pct, 2),
    })

categorical_df = pd.DataFrame(categorical_data)
display(categorical_df)

`Propierty Type`, `Room Type`, `Bed Type` veamos valores únicos e histograma

In [ ]:
# Análisis de Property Type, Room Type, Bed Type con histogramas
import matplotlib.pyplot as plt
import seaborn as sns

variables_categoricas = ['Property Type', 'Room Type', 'Bed Type']

# Configurar el estilo de los gráficos
plt.style.use('default')
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Distribución de Variables Categóricas', fontsize=16, fontweight='bold')

for i, var in enumerate(variables_categoricas):
    print(f"\n{'='*50}")
    print(f"ANÁLISIS DE {var.upper()}")
    print(f"{'='*50}")
    
    # Valores únicos
    valores_unicos = pretrain_data_madrid[var].nunique()
    print(f"Valores únicos: {valores_unicos}")
    
    # Distribución de frecuencias
    print(f"\nDistribución de frecuencias:")
    distribucion = pretrain_data_madrid[var].value_counts()
    print(distribucion)
    
    # Crear histograma
    ax = axes[i]
    distribucion.plot(kind='bar', ax=ax, color='skyblue', alpha=0.7)
    ax.set_title(f'{var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Categorías', fontsize=10)
    ax.set_ylabel('Frecuencia', fontsize=10)
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    ax.tick_params(axis='y', labelsize=8)
    
    # Añadir valores en las barras
    for j, v in enumerate(distribucion.values):
        ax.text(j, v + max(distribucion.values) * 0.01, str(v), 
                ha='center', va='bottom', fontsize=8)
    
    print(f"\n" + "-"*50)

plt.tight_layout()
plt.show()

In [ ]:
pretrain_data.describe().T

In [ ]:
pretrain_data.info()

## 4. Preprocesado datos Train
#### Proceso datos de train en función de las conclusiones estadisticas
- Imputación de valores ausentes
- Codificación variables categoricas
- (Eliminación por random forest/Lasso, muchos Nan, alta correlación..)
- (Generación y transformación de variables)
- (Filtrado outliers)

`Property Type` agrupo por Apartment, House, Condominium, Bed & Breakfast, Loft y el resto todo en Other. Hago un mean encoding con el precio (Esto puedo hacerlo porque ya tengo la división de train y test, y lo estoy haciendo sólo teniendo en cuenta los datos de train). Crear variables dummy generaria demasiadas caracteristicas nuevas.

`First Review`, `Last Review` Voya usar la diferencia en días para tratarla como númerica (y sacar la frecuencia de revisones conjunto a `Number of reviews`más adelante)

In [ ]:
import pandas as pd

pretrain_data_final = pretrain_data.copy() #Train de sólo madrid sin columnas texto con transformaciones aplicadas

# Convertir a datetime
pretrain_data_final['First Review'] = pd.to_datetime(pretrain_data_final['First Review'], errors='coerce')
pretrain_data_final['Last Review'] = pd.to_datetime(pretrain_data_final['Last Review'], errors='coerce')

# Calcular diferencia en días
pretrain_data_final['Review_Days_Diff'] = (pretrain_data_final['Last Review'] - pretrain_data_final['First Review']).dt.days

# Rellenar valores nulos con 0 y aseguro que no haya valores negativos
pretrain_data_final['Review_Days_Diff'] = pretrain_data_final['Review_Days_Diff'].fillna(0)
pretrain_data_final['Review_Days_Diff'] = pretrain_data_final['Review_Days_Diff'].clip(lower=0)

# Nueva columna
print(f"Nueva columna 'Review_Days_Diff' creada:")
print(f"Tipo de datos: {pretrain_data_final['Review_Days_Diff'].dtype}")
print(f"Valores únicos: {pretrain_data_final['Review_Days_Diff'].nunique()}")

print(pretrain_data_final['Review_Days_Diff'].describe())

# Mostrar algunos ejemplos
print(f"\nPrimeros 10 valores:")
print(pretrain_data_final[['First Review', 'Last Review', 'Review_Days_Diff']].head(5))

- `Host Acceptance Rate`, `Has Availability`: puedo eliminarlas porque todos son valores nulos 

- `First Review`, `Last Review`: puedo eliminarlas porque las transforme en `Review_Days_Diff`

- `Zipcode`, `Amenities`,`Propierty Type`, `Room Type`, `Bed Type`: las elimino y no la voy a incluir de momento

In [ ]:
# Eliminar columnas especificadas del dataset pretrain_data
columns_to_remove = [
    'Experiences Offered', 'Host Since', 'Host Acceptance Rate', 'Has Availability',
    'First Review', 'Last Review', 'Zipcode', 'Amenities', 'Property Type', 
    'Room Type', 'Bed Type'
]

# Eliminar las columnas existentes
pretrain_data_final = pretrain_data_final.drop(columns=columns_to_remove)

print(f"\nDataset original: {pretrain_data.shape}")
print(f"Dataset después de eliminar columnas: {pretrain_data_final.shape}")
print(f"Columnas eliminadas: {len(columns_to_remove)}")

# Mostrar las primeras filas del dataset limpio
pretrain_data_final.head().T

In [ ]:
# IMPUTACIÓN SIMPLE 

# Imputar con mediana (más robusta que la media ante outliers)
for col in pretrain_data_final.columns:
    if pretrain_data_final[col].isnull().any():
            median_value = pretrain_data_final[col].median()
            pretrain_data_final[col] = pretrain_data_final[col].fillna(median_value)
            print(f"• {col}: imputado con mediana = {median_value:.2f}")

# Verificar resultado
print(f"\nNulos DESPUÉS:")
nulls_after = pretrain_data_final.isnull().sum()
nulls_after = nulls_after[nulls_after > 0]

if len(nulls_after) == 0:
    print("No quedan valores nulos")
else:
    print("Aún quedan nulos:")
    for col, count in nulls_after.items():
        print(f"• {col}: {count}")



## 5. Procesado datos Test
#### Aplico las mismas transformaciones que en Train

RESUMEN TRANSFORMACIONES EN TRAIN

In [ ]:
#CARGO DATOS TRAIN
pretrain_data = pd.read_csv('airbnb-listings-extract-train.csv', sep=';', decimal='.') #Train de sólo madrid sin columnas texto


#COPIA PARA APLICAR LAS TRANSFORMACIONES
pretrain_data_final = pretrain_data.copy() #Train de sólo madrid sin columnas texto con transformaciones aplicadas


#COLUMNA NUEVA 'Review_Days_Diff'
# Convertir a datetime
pretrain_data_final['First Review'] = pd.to_datetime(pretrain_data_final['First Review'], errors='coerce')
pretrain_data_final['Last Review'] = pd.to_datetime(pretrain_data_final['Last Review'], errors='coerce')
# Calcular diferencia en días
pretrain_data_final['Review_Days_Diff'] = (pretrain_data_final['Last Review'] - pretrain_data_final['First Review']).dt.days
# Rellenar valores nulos con 0 y aseguro que no haya valores negativos
pretrain_data_final['Review_Days_Diff'] = pretrain_data_final['Review_Days_Diff'].fillna(0)
pretrain_data_final['Review_Days_Diff'] = pretrain_data_final['Review_Days_Diff'].clip(lower=0)


#ELIMINO COLUMNAS
# Eliminar columnas especificadas del dataset pretrain_data
columns_to_remove = [
    'Experiences Offered', 'Host Since', 'Host Acceptance Rate', 'Has Availability',
    'First Review', 'Last Review', 'Zipcode', 'Amenities', 'Property Type', 
    'Room Type', 'Bed Type'
]
# Eliminar las columnas existentes
pretrain_data_final = pretrain_data_final.drop(columns=columns_to_remove)


#IMPUTACIÓN NULOS
for col in pretrain_data_final.columns:
    if pretrain_data_final[col].isnull().any():
            median_value = pretrain_data_final[col].median()
            pretrain_data_final[col] = pretrain_data_final[col].fillna(median_value)

In [ ]:
pretrain_data_final.describe().T

APLICO A TEST IGUAL

In [110]:
#CARGO DATOS TEST
pretest_data = pd.read_csv('airbnb-listings-extract-test.csv', sep=';', decimal='.') #Test de sólo madrid sin columnas texto

In [ ]:
#COPIA PARA APLICAR LAS TRANSFORMACIONES
pretest_data_final = pretest_data.copy() #Test de sólo madrid sin columnas texto con transformaciones aplicadas según datos TRAIN


#COLUMNA NUEVA 'Review_Days_Diff'
# Convertir a datetime
pretest_data_final['First Review'] = pd.to_datetime(pretest_data_final['First Review'], errors='coerce')
pretest_data_final['Last Review'] = pd.to_datetime(pretest_data_final['Last Review'], errors='coerce')
# Calcular diferencia en días
pretest_data_final['Review_Days_Diff'] = (pretest_data_final['Last Review'] - pretest_data_final['First Review']).dt.days
# Rellenar valores nulos con 0 y aseguro que no haya valores negativos
pretest_data_final['Review_Days_Diff'] = pretest_data_final['Review_Days_Diff'].fillna(0)
pretest_data_final['Review_Days_Diff'] = pretest_data_final['Review_Days_Diff'].clip(lower=0)


#ELIMINO COLUMNAS
# Eliminar columnas especificadas del dataset pretrain_data
columns_to_remove = [
    'Experiences Offered', 'Host Since', 'Host Acceptance Rate', 'Has Availability',
    'First Review', 'Last Review', 'Zipcode', 'Amenities', 'Property Type', 
    'Room Type', 'Bed Type'
]
# Eliminar las columnas existentes
pretest_data_final = pretest_data_final.drop(columns=columns_to_remove)


#IMPUTACIÓN NULOS (con los resultados de train)
for col in pretest_data_final.columns:
    if pretest_data_final[col].isnull().any():
            median_value = pretrain_data_final[col].median()
            pretest_data_final[col] = pretrain_data_final[col].fillna(median_value)

In [112]:
pretest_data_final.describe().T

,count,mean,std,min,25%,50%,75%,max
Host Response Rate,2651.0,95.574500,14.144084,0.000000,100.000000,100.000000,100.000000,100.000000
Host Listings Count,2651.0,9.540551,26.498575,0.000000,1.000000,2.000000,5.000000,207.000000
Host Total Listings Count,2651.0,9.540551,26.498575,0.000000,1.000000,2.000000,5.000000,207.000000
Latitude,2651.0,40.420510,0.019892,40.332908,40.410018,40.418476,40.427659,40.562736
Longitude,2651.0,-3.697344,0.023357,-3.863907,-3.707892,-3.701861,-3.693721,-3.576484
Accommodates,2651.0,3.195775,1.987640,1.000000,2.000000,2.000000,4.000000,16.000000
Bathrooms,2651.0,1.257450,0.588904,0.000000,1.000000,1.000000,1.000000,8.000000
Bedrooms,2651.0,1.277631,0.786601,0.000000,1.000000,1.000000,1.000000,10.000000
Beds,2651.0,1.958883,1.449856,1.000000,1.000000,1.000000,2.000000,16.000000
Square Feet,2651.0,116.955111,90.528144,0.000000,108.000000,108.000000,108.000000,2153.000000


## 6. Preparacion dataset Train / Test
#### Normalizar Test según Train si aplica

## 7. Modelado
#### Cross validation

## 8. Evaluación